In [ ]:
import os, ffmpy, glob, subprocess, json, dateutil

In [ ]:
class gopro:
    out_options = {'c':'copy'}
    
    lowres_params = {'size':'1280x720',
                     'frame rate': '60',
                     'append':'low',
                     'vcodec':'libx264'}
    
    audio_params = {'quality':'copy',
                    'outformat':'wav',
                    'append':'wav'}
    
    overwrite = False
    testrun = False
    trims   = {}

    _gendict =  {'size':'s',
                 'frame rate':'r',
                 'vcodec':'c:v',
                 'tstart':'ss',
                 'duration':'t',
                 'c':'c',
                 't':'t',
                 'ss':'ss',
                 'c:v':'c:v',
                 'r':'r',
                 's':'s',
                 'append':'',
                 'quality':'c'}
    
    _ignore_keys = ['format', 'append', 'suffix', 'sep']
    
    default_params = {
        'filename':{'append':'',
                    'suffix':'',
                    'sep':'-'
                   },
        'audio':{
            'outformat':'wav'
        },
        'video':{
            'outformat':'mp4'
        }
    }
    
    sep = '-'
    
    fprobe_params = {'print_format':'json',
                     'v':'quiet',
                     'show_entries':'stream=index,codec_type:stream_tags=creation_time:format_tags=creation_time'}
    
    def _filename_basics(self,dictionary):
        out = []
        for key,value in self.default_params['filename'].items():
            if key not in dictionary.keys():
                out.append(value)
            else:
                out.append(dictionary[key])
        
        return out
    
    def _create_outfile(self, infile, outformat, append='', suffix=''):
        basepath = os.path.dirname(infile)
        basename = os.path.basename(infile).split('.')[0]
        
        if append != '':
            append = append + self.sep
        
        if suffix != '':
            suffix = self.sep + suffix
            
        lfile = os.path.join(basepath,append + basename + suffix + '.' + outformat)
        return outfile
    
    def _unpack_dict_to_list(self, dictionary, key_append='-', val_append='', translate=True):
        ulist = []
        for key, value in dictionary.items():
            if key not in self._ignore_keys:
                if translate:
                    key = self._gendict[key]
                temp = [key_append + key,val_append + value]
                ulist = ulist + temp
        return ulist
    
    def _ff(self,indict, outdict, debug=False, run=True):
        
        if debug:
            print('input ', indict)
            print('output', outdict)
        ff = ffmpy.FFmpeg(inputs=indict, outputs=outdict)
        
        if debug or (not self.testrun):
            print('running command \n', ff.cmd)
        if not self.testrun:
            ff.run()
        return

    def silence_video(self, vfile, out=None, debug=False, keep_tmp=False):
        r""" remove audio from given video file. 
        If out is None (default), then it overwrites the original file"""
        
        out_options = ['-an'] + self._unpack_dict_to_list(self.out_options)
        
        tmpf = vfile.split('.')[0]+'-tmp.'+vfile.split('.')[-1]
        
        indict  = {vfile:'-y'}
        outdict = {tmpf:out_options}
        
        self._ff(indict,outdict)
                
        os.remove(vfile)
        if not keep_tmp:
            print('remove tmp')
            #os.rename(src=tmpf, dst=vfile)
        return
    
    def resize_video(self, vfile, keep_highres=True, debug=False):
        
        #for key in self.lowres_params.keys()
        
        indict = {vfile:'-y'}
        
        basepath = os.path.dirname(vfile)
        basename = os.path.basename(vfile)
        
        if 'append' in self.lowres_params:
            addname = self.lowres_params['append']
        else:
            addname = 'low'
            
        lfile = os.path.join(basepath,addname + '-' + basename)
        
        out_options = ['-an'] + self._unpack_dict_to_list(self.lowres_params)
        
        outdict = {lfile:out_options}
        
        self._ff(indict,outdict, debug=debug)
        
        return
    
    def audio_convert(self, afile, debug=False):
        indict = {afile:'-y'}
        
        #filename_params = self._filename_basics(self.audio_params)
        outformat = self.audio_params['outformat']
        append = self.audio_params['append']
        suffix = self.audio_params['suffix']
        
        outfile = self._create_outfile(afile, outformat, append=append, suffix=suffix)
        
        outdict = {outfile:self._unpack_dict_to_list(self.out_options)}
        
        self._ff(indict,outdict, debug=debug)
        
        return
    
    def audio_extract(self, vfile, debug=False):
        indict = {vfile:'-y'}
        
        basepath = os.path.dirname(afile)
        basename = os.path.basename(afile).split('.')[0]
        lfile = os.path.join(basepath,append + '-' + basename + '.' + out_format)
    
    def trim_videos(self,video_path, trims, options,
                    input_format = None, output_format = None,
                    append_folder = False, join_cuts = False, 
                    debug = False, dryrun = False):
        fpath = video_path
        optlist = self._unpack_dict_to_list(options, translate=False)
        
        if join_cuts:
                fjoin = open(os.path.join(fpath,'concat.txt'),'w')
                foutc = os.path.join(fpath,'cuts-combined')
                files = []
        for key in trims.keys():
            try:
                fbase = key.split('.')[0]
            except:
                fbase = key

            if input_format is None:
                input_format = trims[key]['input_format']

            if output_format is None:
                output_format = trims[key]['output_format']

            append_filename = ''
            if 'append' in trims[key]:
                append_filename = trims[key]['append']
            if append_folder:
                append_filename = d + '-' + append_filename

            fname_in  = glob.glob(os.path.join(fpath,fbase+'*.'+input_format))[0]
            fname_out = os.path.join(fpath,append_filename + '-' + fbase + '.' + output_format)

            if trims[key]['audio'] is not '':
                opts = ['-an'] + optlist

            trimopts = ['-ss', trims[key]['tstart'],
                    '-t',trims[key]['duration']] 
            outopts  = trimopts+ opts

            if output_format.lower() == 'wmv':
                outopts = ['-qscale', '0'] + outopts

            if (d is not '') and (d in flipvidsinfolders):
                outopts = ['-vf', 'transpose=1'] + outopts
                print(d,' rotate')

            indict  = {fname_in:'-y'}
            outdict = {fname_out:outopts}

            ff[key] = ffmpy.FFmpeg(inputs=indict, outputs=outdict)
        for key in ff.keys():
            print(ff[key].cmd+'\n')
            print('running ffmpeg...')
            if not dryrun:
                ff[key].run()
            print('done \n')

In [ ]:
basepath = '/Volumes/media/videos/gopro/BG-00007/side-C/'

# create a dictionary with the file names and start ending times - this defines the actions of trim in every directory
trims ={
    '10222018-1417.0':{'tstart':'00:05:45',
                'duration':'00:01:00',
                'append':'cut-1',
                'audio':None
               },
    '10222018-1417.1':{'tstart':'00:07:00',
                'duration':'00:01:45',
                'append':'cut-2',
                'audio':None
               },
    '10222018-1425.0':{'tstart':'00:00:00',
                'duration':'00:01:00',
                'append':'cut-1',
                'audio':None
               }
#     'GX020010.2':{'tstart':'00:01:45',
#                 'duration':'00:03:15',
#                 'append':'action',
#                 'audio':'split'
#                }
}

options = {
    'r':'60',
    's':'1280x720',
    'c:v':'libx264',
    'pix_fmt':'yuv420p',
    'preset':'medium'
}



cut_join = False
cuts_rm  = False
append_folder = True
input_format  = 'MP4'
output_format = 'mp4'
#flipvidsinfolders = ['camera1']

In [ ]:
basepath = '/Volumes/extreme/repos/lab-data/laser-experiment/scale_removal/ScCBs_00001/video/'
trims = {
    'CBs_00001_04302018-1.0':{
        'tstart':'00:00:21',
        'duration':'00:03:03',
        'append':'cut-1',
        'audio':None
    },
}

options = {
    'r':'60',
    's':'1280x720',
    'c:v':'libx264',
    'pix_fmt':'yuv420p',
    'preset':'medium'
}



cut_join = False
cuts_rm  = False
append_folder = False
input_format  = 'MP4'
output_format = 'mp4'

In [ ]:
# basepath = '/Volumes/media/videos/gopro/water-tank/10252018/test-01/'

# # create a dictionary with the file names and start ending times - this defines the actions of trim in every directory
# trims ={
#     'GX040018.0':{'tstart':'00:01:20',
#                   'duration':'00:00:40',
#                   'append':'cut-1',
#                   'audio':None,
#                }
# }

# options = {
#     'r':'60',
#     's':'1280x720',
#     'c:v':'libx264',
#     'pix_fmt':'yuv420p',
#     'preset':'medium'
# }


# cut_join = False
# cuts_rm  = False
# append_folder = True
# input_format  = 'MP4'
# output_format = 'mp4'
# flipvidsinfolders = ['camera1', 'camera2']

In [ ]:
gp = gopro()

In [ ]:
for a,b,c in os.walk(basepath):    
    if len(b) == 0:
        b = ['']
    for d in b:
        fpath = os.path.join(basepath,d)
        gp.trim_videos(fpath,trims, options, input_format='MP4', output_format='mp4',dryrun=True)

In [ ]:
optlist = []
ff = {}
split  = False

for key, value in options.items():
    temp = ['-'+key,value]
    optlist = optlist + temp

for a,b,c in os.walk(basepath):    
    if len(b) == 0:
        b = ['']
    for d in b:
        fpath = os.path.join(basepath,d)
        if cut_join:
            fjoin = open(os.path.join(fpath,'concat.txt'),'w')
            foutc = os.path.join(fpath,'cuts-combined')
            files = []
        for key in trims.keys():
            try:
                fbase = key.split('.')[0]
            except:
                fbase = key
            
            if input_format is None:
                input_format = trims[key]['input_format']
            
            if output_format is None:
                output_format = trims[key]['output_format']
            
            append_filename = ''
            if 'append' in trims[key]:
                append_filename = trims[key]['append']
            if append_folder:
                append_filename = d + '-' + append_filename
            
            fname_in  = glob.glob(os.path.join(fpath,fbase+'*.'+input_format))[0]
            fname_out = os.path.join(fpath,append_filename + '-' + fbase + '.' + output_format)
            
            if trims[key]['audio'] is not '':
                opts = ['-an'] + optlist

            trimopts = ['-ss', trims[key]['tstart'],
                    '-t',trims[key]['duration']] 
            outopts  = trimopts+ opts
            
            if output_format.lower() == 'wmv':
                outopts = ['-qscale', '0'] + outopts
            
            if (d is not '') and (d in flipvidsinfolders):
                outopts = ['-vf', 'transpose=1'] + outopts
                print(d,' rotate')
            
            indict  = {fname_in:'-y'}
            outdict = {fname_out:outopts}

            ff[key] = ffmpy.FFmpeg(inputs=indict, outputs=outdict)
        for key in ff.keys():
            print(ff[key].cmd+'\n')
            print('running ffmpeg...')
            ff[key].run()
            print('done \n')

In [ ]:
len(b)

In [ ]:
optlist = []
ff = {}
split = False

dirs = os.listdir(basepath)

for key, value in options.items():
    temp = ['-'+key,value]
    optlist = optlist + temp

for d in dirs:
    fpath = os.path.join(basepath,d)
    if cut_join:
        fjoin = open(os.path.join(fpath,'concat.txt'),'w')
        foutc = os.path.join(fpath,'cuts-combined')
        files = []
    for key in trims.keys():
        fbase = key #key.split('.')[0]
        #opts = optlist

        fin  = os.path.join(fpath,fbase+'.MP4')
        fout = os.path.join(fpath,trims[key]['append']+'-'+fbase+'.MP4')
        if trims[key]['audio'] is not '':
            opts = ['-an'] + optlist

        trimopts = ['-ss', trims[key]['tstart'],
                '-t',trims[key]['duration']] 
        outopts  = trimopts+ opts

        indict  = {fin:'-y'}
        outdict = {fout:outopts}

        ff[key] = ffmpy.FFmpeg(inputs=indict, outputs=outdict)

        if trims[key]['audio'] is 'split':
            split_audio = True
            faudio  = fout[:-3]+'m4a'
            optsaudio = trimopts + ['-map','0:1'] + optlist
            outdict = {faudio:optsaudio}
            ff[key+'-audio'] = ffmpy.FFmpeg(inputs=indict, outputs=outdict)
        if (cut_join) and (trims[key]['append'] is 'cut'):
            files.append(fout)
            line = 'file '+ '\'' + fout + '\'' + '\n'
            #print(line)
            fjoin.writelines(line)
            foutc = foutc + '-' + fbase

    for key in ff.keys():
        print(ff[key].cmd+'\n')
        print('running ffmpeg...')
        ff[key].run()
        print('done \n')

    if cut_join:
        fjoin.close()
        foutc = foutc + '.MP4'
        indict = {fjoin.name:['-y', '-f', 'concat', '-safe', '0']}
        outdict = {foutc:optlist}
        ffc = ffmpy.FFmpeg(inputs=indict, outputs=outdict)
        print(ffc.cmd+'\n')
        print('combining cuts...')
        ffc.run()
        print('done \n')

        if cuts_rm:
            if os.path.exists(fjoin.name):
                os.remove(fjoin.name)
            for file in files:
                if os.path.exists(file):
                    os.remove(file)
                print(file)

In [ ]:
try: 
    del gp
except:
    pass
gp = gopro()
files = glob.glob(os.path.join(basepath,'*','10222018-1417*GX*.MP4'))
print(files, sep='\n')
# for vfile in files:
#     gp.silence_video(vfile, debug=True, keep_tmp=True)

In [ ]:
gp = gopro()
files = glob.glob(os.path.join(basepath,'*','*.MP4'))
for file in files:
    print(file)
    #gp.testrun = True
    gp.resize_video(file,debug=True)
    #os.rename(src=file, dst=file.replace('-tmp',''))

In [ ]:
gp = gopro()
files = glob.glob(os.path.join(basepath,'*','*.m4a'))
for file in files:
    print(file)
    #gp.testrun = True
    gp.audio_convert(file,debug=True)
    #os.rename(src=file, dst=file.replace('-tmp',''))